In [6]:
import argparse
import os
import random
import sys
sys.path.append('../')
from poker_env_multi import PokerEnvMulti
from agents.random_policy import RandomActions
from ray.rllib.algorithms.ppo import PPOConfig

import ray
from ray import air, tune
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from ray.rllib.algorithms.pg import (
    PG,
    PGConfig,
    PGTF2Policy,
    PGTF1Policy,
    PGTorchPolicy,
)

from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import register_env

In [7]:
def env_creator(config):
    env = PokerEnvMulti(config)
    return env

register_env("poker", lambda config: env_creator(config))

In [8]:
def select_policy(agent_id, episode, **kwargs):
    #if agent_id == "player_0":
    #    return "learned"
    #else:
    #    return random.choice(["always_same", "beat_last"])
    return "random"

config = (
    PPOConfig()
    .rollouts(
        num_rollout_workers=0,
        num_envs_per_worker=1,
    )
    .training(
        train_batch_size=128,
        gamma=0.99,
    )
    .multi_agent(
        policies={
            "random": PolicySpec(policy_class=RandomActions),
            #"learned": PolicySpec(
            #    config={
            #        "model": {"use_lstm": use_lstm},
            #        "framework": args.framework,
            #    }
            #),
        },
        policy_mapping_fn=select_policy,
        policies_to_train=[],
    )
    .resources(num_gpus=0)\
)

trainer = config.build(env="poker")


2022-10-31 17:49:44,309	WARNING multi_agent.py:121 -- `config.multiagent.policies_to_train` is empty! Make sure - if you would like to learn at least one policy - to add its ID to that list.
2022-10-31 17:49:44,310	WARNING ppo.py:350 -- `train_batch_size` (128) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 128.
2022-10-31 17:49:44,328	WARNING util.py:66 -- Install gputil for GPU system monitoring.


In [9]:
trainer.train()

{3: 1}
6
{4: 2}
6
{5: 2}
6
{0: 1}
6
{1: 3}
6
{2: 3}
6
{3: 1}
6
{4: 0}
5
{5: 3}
5
{0: 3}
5
{1: 2}
5
{2: 1}
5
{3: 1}
5
{5: 1}
5
{0: 0}
progress
4
{1: 3}
4
{2: 3}
4
{3: 3}
4
{5: 3}
progress
4
{1: 1}
4
{2: 3}
4
{3: 2}
4
{5: 2}
4
{1: 1}
4
{2: 1}
4
{3: 3}
4
{5: 2}
4
{1: 1}
4
{2: 1}
4
{3: 3}
4
{5: 2}
4
{1: 3}
4
{2: 3}
4
{3: 2}
4
{5: 3}
4
{1: 2}
4
{2: 2}
4
{3: 3}
4
{5: 2}
4
{1: 3}
4
{2: 2}
4
{3: 0}
3
{5: 2}
3
{1: 0}
progress
2
{2: 3}
2
{5: 3}
progress
showdown
2
done
{'hands': ['EIGHT of HEARTS; KING of CLUBS; ', 'TWO of HEARTS; TEN of CLUBS; ', 'FIVE of SPADES; SIX of DIAMONDS; ', 'KING of HEARTS; FIVE of CLUBS; ', 'SEVEN of SPADES; QUEEN of DIAMONDS; ', 'JACK of HEARTS; THREE of DIAMONDS; '], 'community': 'NINE of CLUBS; SEVEN of DIAMONDS; ACE of CLUBS; TEN of SPADES; ACE of HEARTS; ', 'scores': array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 1.,  0.,  9.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 1.,  0., 10.]]), 'winners': 5}
{4: 0}
5
{5: 0}
4
{0: 1}
4
{1: 3}
4
{2:

{'custom_metrics': {},
 'episode_media': {},
 'num_recreated_workers': 0,
 'info': {'learner': {},
  'num_env_steps_sampled': 128,
  'num_env_steps_trained': 128,
  'num_agent_steps_sampled': 123,
  'num_agent_steps_trained': 123},
 'sampler_results': {'episode_reward_max': 8.0,
  'episode_reward_min': 0.0,
  'episode_reward_mean': 4.833333333333333,
  'episode_len_mean': 20.0,
  'episode_media': {},
  'episodes_this_iter': 6,
  'policy_reward_min': {'random': -8.0},
  'policy_reward_max': {'random': 26.0},
  'policy_reward_mean': {'random': 0.8055555555555556},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [0.0, 8.0, 4.0, 8.0, 1.0, 8.0],
   'episode_lengths': [46, 10, 13, 24, 15, 12],
   'policy_random_reward': [-2.0,
    26.0,
    -2.0,
    -8.0,
    -6.0,
    -8.0,
    0.0,
    10.0,
    -2.0,
    0.0,
    0.0,
    0.0,
    -1.0,
    0.0,
    -1.0,
    7.0,
    -1.0,
    0.0,
    8.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    -3.0,
    11.0,
    -3.0,
  

In [10]:
import numpy as np
winners = np.array([0,2])
losers = np.delete(np.arange(6), winners)
losers

array([1, 3, 4, 5])